In [1]:
# import importlib
# import db_utils
# importlib.reload(db_utils)


In [2]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms.ollama import Ollama

from db_utils import add_documents_to_db, remove_index, remove_document, fetch_all
from files_utils import chunk_documents
from model_utils import rag_prompt

Prepare chunks

In [7]:
pdf_directory = "./data"

document_chunks = chunk_documents(pdf_directory=pdf_directory)

In [8]:
document_chunks = document_chunks[50:75] # limit chunks

In [9]:
document_chunks

[Document(page_content='nagiego stosunku pieniężnego.\nBurżuazja ujawniła,  że brutalne stosowanie  siły, które reakcja tak podziwia w średniowieczu, \nznajdowało  godn e uzupełnienie w najbardzie j gnuśnym  próżniactwie . Dopiero burżuazja  pokazała, co może', metadata={'source': 'data/4.-kapitalizm_marks_engels_manifest-komunistyczny.pdf', 'page': 4, 'id': 'data/4.-kapitalizm_marks_engels_manifest-komunistyczny.pdf:4:2'}),
 Document(page_content='sprawić  ludzka działal ność. Dokonał a ona całkiem innych cudów niż zbudowanie egipskich piramid, \nrzymskich wodociągów i gotyckich katedr,  odbyła pochody zgoła inne niż wędrówki ludów i wyprawy \nkrzyżowe.', metadata={'source': 'data/4.-kapitalizm_marks_engels_manifest-komunistyczny.pdf', 'page': 4, 'id': 'data/4.-kapitalizm_marks_engels_manifest-komunistyczny.pdf:4:3'}),
 Document(page_content='Burżuazja nie może istnieć bez nieustannego  rewolucjonizowani a narzędzi produkcji, a więc \nstosunków produkcji, a więc całokształtu stosunków

Prepare database

In [10]:
embedding = OllamaEmbeddings(model='mistral-7b-instruct-v0.1.Q5_K_M:latest')

db_kwargs = {
    "embedding": embedding,
    "es_url": "http://localhost:9200",
    "index_name": "rag",
    "distance_strategy": "COSINE"
}

# remove existing index if needed
remove_index(index_name=db_kwargs["index_name"], db_config={"hosts": db_kwargs["es_url"]})

db = add_documents_to_db(db=None, document_chunks=document_chunks,  db_kwargs=db_kwargs)

Index 'rag' deleted successfully.


Ingesting documents: 100%|██████████| 25/25 [03:12<00:00,  7.69s/it]


In [11]:
fetch_all(
    index_name=db_kwargs["index_name"],
    db_config={"hosts": db_kwargs["es_url"]},
)

Retrieved 25 documents.
{'text': 'pogląda mi ulegają rozkładow i, wszystkie  nowo powstałe  stają się przestarzałe,  zanim zdążą  skostnieć. \nWszystko, co stanowe i zakrz epłe, znika, wszy stko, co święte, ulega sprofanow aniu i ludzie muszą wreszcie', 'metadata': {'source': 'data/4.-kapitalizm_marks_engels_manifest-komunistyczny.pdf', 'page': 4, 'id': 'data/4.-kapitalizm_marks_engels_manifest-komunistyczny.pdf:4:7'}}
{'text': 'narodo wy. Odwieczne narodo we gałęzie przemysłu uległy zniszczeniu i są dalej co dzień niszczone. Są \nwypierane przez nowe gałęz ie przemysłu, których wprowadzeni e staje się kwestią życia dla wszystk ich', 'metadata': {'source': 'data/4.-kapitalizm_marks_engels_manifest-komunistyczny.pdf', 'page': 4, 'id': 'data/4.-kapitalizm_marks_engels_manifest-komunistyczny.pdf:4:10'}}
{'text': 'nagiego stosunku pieniężnego.\nBurżuazja ujawniła,  że brutalne stosowanie  siły, które reakcja tak podziwia w średniowieczu, \nznajdowało  godn e uzupełnienie w najbardzie j gnu

Prompt rag

In [12]:
model = Ollama(model='mistral-7b-instruct-v0.1.Q5_K_M:latest')

response = rag_prompt(
    query = "Czemu winna jest burżuazja?",
    model=model,
    db=db,
)

# remove_document(
#     index_name=db_kwargs["index_name"],
#     source_file="",
#     db_config={"hosts": db_kwargs["es_url"]},
#     )


Human: 
    Odpowiedz na pytanie bazując tylko na poniższym kontekście:

    zniewala  wszy stkie narody  do przysw ojenia sobie burżuazyjnego  sposobu produkcji, zniewala  je do 
wprowadzenia  u siebie tak zwanej cywilizacji, tzn. do stania się burżua. Słowem, stwarz a sobie świat na 
obraz i podobieństwo swoje.

---

duchowej. Wytwo ry duchowe poszczególnych narodó w stają  się wspólnym  dobrem. Jednostronność i 
ograniczoność narodowa staje się coraz bardzie j niemożliwa, a z wielu literatur narodowych i regionalnych 
powstaje literatura światow a.

---

sprawić  ludzka działal ność. Dokonał a ona całkiem innych cudów niż zbudowanie egipskich piramid, 
rzymskich wodociągów i gotyckich katedr,  odbyła pochody zgoła inne niż wędrówki ludów i wyprawy 
krzyżowe.

---

ludności z idiotyzm u życia wiejskiego. Podobnie jak uzależniła wieś od miast a, tak samo  uzależniła kraje 
barbarzyńskie  i półbarbarzyński e od krajów  cywilizowanych,  narody  chłopskie od narodów burżuazyjnych, 
Wschó